In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

#replace variables with values for your account
payload = {
    'client_id': "106601",
    'client_secret': '9a38b8e0bf43bb03eade5c23d9c8c3711318c0b2',
    'refresh_token': 'd52abdcad86ebdd81a63936ad1d929d859c5bd58',
    'grant_type': "refresh_token",
    'f': 'json'
}

In [5]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))


Requesting Token...

Access Token = 68c478e9f54ff5b3886ae2d225d6c539f8cd1690



In [10]:
# Define activites_url
activites_url = 'https://www.strava.com/api/v3/athlete/activities'

# Ensure access_token is properly obtained
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}

try:
    # Make the API request
    response = requests.get(activites_url, headers=header, params=param)
    
    # Check if request was successful
    response.raise_for_status()
    
    # Parse JSON response
    my_dataset = response.json()
    
    # Check if dataset is a list and not empty
    if isinstance(my_dataset, list) and len(my_dataset) > 0:
        first_activity = my_dataset[0]
        if "name" in first_activity and "map" in first_activity and "summary_polyline" in first_activity["map"]:
            print(first_activity["name"])
            print(first_activity["map"]["summary_polyline"])
        else:
            print("Required keys not found in the response")
    else:
        print("No data found in the response or response is not a list")
        
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")
except KeyError as e:
    print(f"Key error: {e} not found in the response")
except IndexError:
    print("Dataset is empty or index out of range")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Request error: 401 Client Error: Unauthorized for url: https://www.strava.com/api/v3/athlete/activities?per_page=200&page=1


In [11]:
import requests

# Proper header formatting
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Make authenticated request
response = requests.get(url, headers=headers)

# Check for 401 status
if response.status_code == 401:
    # Request new access token
    # Retry request with new token


SyntaxError: incomplete input (3750058806.py, line 15)

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

ImportError: cannot import name 'json_normalize' from 'pandas.io.json' (c:\Users\schol\anaconda3\Lib\site-packages\pandas\io\json\__init__.py)

In [ ]:
activities = json_normalize(my_dataset)

In [ ]:
#Explore dataset

#activities.columns

In [ ]:
#Create new dataframe with only columns I care about
cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
         'average_speed', 'max_speed','total_elevation_gain',
         'start_date_local'
       ]

activities = activities[cols]

#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date

activities.head(5)

In [ ]:
activities['type'].value_counts()

In [ ]:
#download data
#activities.to_csv('activities.csv',index=False)

In [ ]:
#graphs

In [ ]:
import seaborn as sns

In [ ]:
runs = activities.loc[activities['type'] == 'Run'] 
runs.head(5)

In [ ]:
sns.set(style="ticks", context="talk")
sns.scatterplot(x='distance', y = 'average_speed', data = runs).set_title("Average Speed vs Distance")


In [ ]:
sns.scatterplot(x='distance', y = 'max_speed', data = runs).set_title("Max Speed vs Distance")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#max speed over time
fig = plt.figure()
ax = fig.add_subplot(111)
x = np.asarray(runs.start_date_local)
y = np.asarray(runs.max_speed)
ax.plot_date(x, y)
ax.set_title('Max Speed over Time')
fig.autofmt_xdate(rotation=45)
fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
x = np.asarray(runs.start_date_local)
y = np.asarray(runs.average_speed)
ax1.plot_date(x, y)
ax1.set_title('Average Speed over Time')
fig.autofmt_xdate(rotation=45)
fig.tight_layout()
fig.show()

In [ ]:
#Bike rides faster in Rockport or DC?

In [ ]:
rides = activities.loc[activities['type'] == 'Ride'] 
rides.head(5)

In [ ]:
from datetime import datetime

In [ ]:
#add home / dc classification 2020-06-27 is date switch
dc_date= '2020-06-27'
dc_date = datetime.strptime(dc_date, '%Y-%m-%d').date()
rides['start_date_local'] = pd.to_datetime(rides['start_date_local']).dt.date
rides['dc'] = np.where(rides['start_date_local'] < dc_date, 'true', 'false')



In [ ]:
dc = rides.loc[rides['dc'] == 'true']
home = rides.loc[rides['dc'] == 'false']
dc_speed = round(dc['average_speed'].mean() * 2.237, 2)
home_speed = round(home['average_speed'].mean() * 2.237, 2)
dc_max_speed = round(dc['max_speed'].mean()* 2.237, 2)
home_max_speed = round(home['max_speed'].mean()* 2.237, 2)
print("Average DC Speed: " + str(dc_speed) + " | Average DC Max Speed: " + str(dc_max_speed) + '\n'
      + "Average Home Speed: " + str(home_speed) + " | Average Home Max Speed: " + str(home_max_speed))


#time is in seconds, distance is in meters
#m/s to mph 1m/s to 2.237 mph

In [ ]:
percent_increase_average = round((home_speed - dc_speed) * 100 / dc_speed,2)
percent_increase_average


In [ ]:
percent_increase_average_max = round((home_max_speed - dc_max_speed) * 100 / dc_max_speed,2)
percent_increase_average_max 

In [ ]:
#elevation change vs average speed

In [ ]:
sns.scatterplot(x='total_elevation_gain', y = 'average_speed', data = rides).set_title("Speed vs Elevation Gain")